# Chapter 1 - Graph Theory

## Requirements

This notebook was tested on Linux and MacOS, using a conda environment.

Details to build the conda environment can be found here: https://github.com/ftheberge/GraphMiningNotebooks

In case users would find some issues with the notebook, we ask to open an issue in the GitHub repository.

For the data, make sure you have the correct directory in next cell.

In [ ]:
## setting the path to the datasets
datadir='../Datasets/'


In [ ]:
## required packages for this Chapter
import igraph as ig
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from collections import Counter
from statsmodels.distributions.empirical_distribution import ECDF as ecdf


## Summary

In this notebook, we look at **simple graph statistics** such as:
- degree distribution
- clustering coefficient
- shortest path length distribution

We illustrate the fact that even with such simple measures, we can see big difference between different types of graphs, thus the importance of **EDA**, exploratory data analysis, as a first step in data mining. 

In this notebook we consider two graphs:

- a social-type graph, links between **GitHub developers**, and
- a transportation-type network, namely a **power grid**.

## GitHub Developers Graph

#### Description

A large undirected social network of GitHub developers which was collected from the public API in June 2019. Nodes are developers who have starred at least 10 repositories and edges are mutual follower relationships between them. The vertex features are extracted based on the location, repositories starred, employer and e-mail address. 

The graph has two types of nodes: 
- web developer 
- machine learning developer 

Below, we construct this graph (with igraph); later we will look at subgraphs respectively for web or machine learning developers.

The graph is stored in object ```GitHubGraph```; look at the code below to see how we can also store attributes for edges and vertices in igraph objects.

There are several ways to read the edges; here we used a function from the ```pandas``` package.


In [ ]:
## read the GitHub edge list as tuples and build undirected graph
## each node index is stored in vertex attribute "id"
df = pd.read_csv(datadir+'GitHubDevelopers/musae_git_edges.csv')
GitHubGraph = ig.Graph.TupleList([tuple(x) for x in df.values], directed = False, vertex_name_attr='id')


In [ ]:
## read node attributes
Attr = pd.read_csv(datadir+'GitHubDevelopers/musae_git_target.csv')
Attr.head()

In [ ]:
## build attribute dictionaries
Names = dict(zip(Attr.id,Attr.name))
ML = dict(zip(Attr.id,Attr.ml_target))

In [ ]:
## add name attributes to graph
GitHubGraph.vs['name'] = [Names[i] for i in GitHubGraph.vs['id']]

## add a class: 'ml' or 'web' depending on attribute 'ml_label'
labels = ['web','ml']
GitHubGraph.vs['class'] = [labels[ML[i]] for i in GitHubGraph.vs['id']]


In [ ]:
## set node colors depending on ml label
## also set actual label ('ml' or 'web')
cls = {'web':'grey','ml':'black'} ## pick colors
GitHubGraph.vs['color'] = [cls[i] for i in GitHubGraph.vs['class']]

## set edge color
GitHubGraph.es['color'] = 'grey'

### GitHub subgraphs

We build the two subgraphs below: ```subgraph_ml``` for the machine learning (**ml**) developers and ```subgraph_web``` for the **web** developers.

In [ ]:
## build the subgraphs
subgraph_ml = GitHubGraph.subgraph([v for v in GitHubGraph.vs() if v['class']=='ml'])
subgraph_web = GitHubGraph.subgraph([v for v in GitHubGraph.vs() if v['class']=='web'])

## there are 9739 ml developers and 27961 web developers
print('GitHub nodes:',GitHubGraph.vcount(),'\nml developers:',subgraph_ml.vcount(),
      '\nweb developers:',subgraph_web.vcount())


Note that some ml developers are connected only to web developers and vice-versa. 

Therefore, some nodes will end up with no connection (degree 0) in the subgraphs.


In [ ]:
## github graph: count 'ml' nodes with edges to 'web' nodes, and vice-versa
count_ml = count_web = 0
for v in GitHubGraph.vs():
    if v['class']=='ml':
        ## looking at all neighbors as a 'set' for this test:
        if set([GitHubGraph.vs[i]['class'] for i in GitHubGraph.neighbors(v)])=={'web'}:
            count_ml+=1
    else:
        if set([GitHubGraph.vs[i]['class'] for i in GitHubGraph.neighbors(v)])=={'ml'}:
            count_web+=1
print(count_ml,'ml developers connected only to web developers\n',
      count_web,'web developers connected only to ml developers')        

In [ ]:
## this should correpond to the number of degree 0 nodes in the subgraphs,
## as there are no degree 0 nodes in the original graph.
print('nodes of degree zero in overall graph:',sum([d==0 for d in GitHubGraph.degree()]))
print('nodes of degree zero in ml subgraph:',sum([i==0 for i in subgraph_ml.degree()]))
print('nodes of degree zero in web subgraph:',sum([i==0 for i in subgraph_web.degree()]))


## Europe Electric Grid

A network of high voltage grid in Europe. The vertices are stations and the edges are the lines connecting them.
More details can be found at: https://zenodo.org/record/47317#.Xt6nzy3MxTY.

The graph is stored in object ```Grid```.

The edges have directionality, but for this notebook, we consider an undirected version of this graph, so after reading the edges, we ''simplify'' the graph to remove multiedges.

Nodes have different attributes, including longitude and latitude; we will use those to force a graph layout for plotting according to the geographical location.

There are several types of nodes: 'joint', 'merge', 'plant', 'station', 'sub_station', 'substation'.

We show a different wat to read in edges and build a graph

In [ ]:
## read edge list for the grid network and build undirected simple graph
## by default, node ids are saves in the 'name' attribute as string
Grid = ig.Graph.Read_Ncol(datadir+'GridEurope/gridkit_europe-highvoltage.edges', 
                          directed=False)
Grid = Grid.simplify()


In [ ]:
## read the vertices and their attributes
## nb: we are re-using the name 'Attr' here as notebooks are usually run in ordered sequence, 
##     and we no longer need this variable name for the earlier graph.
Attr = pd.read_csv(datadir+'GridEurope/gridkit_europe-highvoltage.vertices')
Attr.head()

In [ ]:
## build attribute dictionaries
Longitude = dict(zip(Attr.v_id,Attr.lon))
Latitude = dict(zip(Attr.v_id,Attr.lat))
Type = dict(zip(Attr.v_id,Attr.typ)) 

In [ ]:
## map names to integers for easier referencing
Grid.vs['id'] = [int(x) for x in Grid.vs['name']]

## save longitude and latitude in the graph 
Grid.vs['longitude'] = [Longitude[i] for i in Grid.vs['id']]
Grid.vs['latitude'] = [Latitude[i] for i in Grid.vs['id']]
Grid.vs['type'] = [Type[i] for i in Grid.vs['id']]

In [ ]:
## EDA - min/max latitude and longitude and node types
print('Node types and counts:')
print(Counter(Grid.vs['type']).most_common())

print('\nLatitude range:',min(Grid.vs['latitude']),max(Grid.vs['latitude']))
print('\nLongitude range:',min(Grid.vs['longitude']),max(Grid.vs['longitude']))


In [ ]:
## ploting the grid - colors w.r.t. node types
## we use negative latitude die to location of the origin in the plot.
Grid.vs['size'] = 4
cls = {'joint':'pink', 'substation':'lightblue', 'merge':'brown', 'station':'blue', 
       'sub_station':'lightblue', 'plant':'black'}
Grid.vs['color'] = [cls[x] for x in Grid.vs['type']]
Grid.es['color'] = 'grey'
Grid.layout = [(v['longitude'],-v['latitude']) for v in Grid.vs]
ig.plot(Grid, layout=Grid.layout, bbox=(0,0,600,450))

# Graph Features

Below, we compute several basic features for the 4 graphs we have:
- GitHub graph (ml+web)
- GitHub ml developers subgraph
- GitHub web developers subgraph
- Grid

Please refer to the book for details of those features.

Note that running the cell below can take a few minutes.


In [ ]:
## compute and store basic statistics in a dataframe
## this can take a few minutes
def baseStats(G):
    deg = G.degree()
    return [G.vcount(),G.ecount(),np.min(deg),np.mean(deg),np.median(deg),np.quantile(deg,.99),
            np.max(deg),G.diameter(),np.max(G.connected_components().membership)+1,
            G.connected_components().giant().vcount(),
            sum([x==0 for x in G.degree()]),G.transitivity_undirected(mode='nan'),
            G.transitivity_avglocal_undirected(mode='nan')]

S = []
S.append(['GitHub']+baseStats(GitHubGraph))
S.append(['GitHub (ml)']+baseStats(subgraph_ml))
S.append(['GitHub (web)']+baseStats(subgraph_web))
S.append(['Grid']+baseStats(Grid))
df = pd.DataFrame(S,columns=['network','# nodes','# edges',r'$\delta$',r'$\langle k\rangle$',r'median degree',
                            r'$d_{quant_{99}}$',r'$\Delta$','diameter','# components','largest component',
                            '# isolates',r'$C_{glob}$',r'$C_{loc}$']).transpose()
df

### Analysis of the results

What do we see in the table above?

First, look at the **degree distribution**; the GitHub graphs have a wide range of values, including some very high degree nodes, while the Grid graph has degree in range 1 to 16 only.

The **diameter** (maximum shortest path length) is also quite different; it is common for social networks to have relatively small diameter. On the other hand, a multi-country power grid has large diameter.

Looking at **components**, the GitHub graph is connected (single component), but the two subgraphs are not, and there are even nodes with null degree as we already saw. The Grid graph has several components (some islands), but most nodes fall is what we call the "giant component".

Finally, we see some differences between the local and global **clustering coefficients** for the GitHub graphs; why is this so? what happens with very high degree nodes?

## Visualize part of the Grid network

For the Grid graph, we select a range of lat/lon that correspond to the Iberic peninsula.



In [ ]:
## subgraph of Grid -- Iberic Peninsula
V = [v for v in Grid.vs() if v['latitude']>36 and v['latitude']<44 and v['longitude']>-10 and v['longitude']<4]
Grid_IP = Grid.subgraph(V)
layout = [(v['longitude'],-v['latitude']) for v in Grid_IP.vs]
ig.plot(Grid_IP, layout=layout, bbox=(0,0,300,250), vertex_color='black', vertex_size=3)


## Visualize part of the GitHub (ml) graph

There is no lat/lon here; in the code below, we take the giant component for the GitHub ml subgraph,
and arbitrary cut w.r.t. the computed layout to display a portion of the subgraph.

This is for illustration only; note that this is quite different from the Grid graph, with clumps (dense areas) and less regular edge distribution.


In [ ]:
## plot subgraph for github(ml)
sg = subgraph_ml.connected_components().giant()
ly = sg.layout_auto()
sg.vs['x'] = [x[0] for x in ly]
sg.vs['y'] = [x[1] for x in ly]

## 
z = 1 
V = []
while(len(V)<400):
    V = [v for v in sg.vs() if v['x']<z and v['x']>-z and v['y']<z and v['y']>-z]
    z += 1
ssg = sg.subgraph(V).connected_components().giant()
print('showing',ssg.vcount(),'nodes')
ig.plot(ssg, bbox=(0,0,300,300), vertex_size=3)

## Degree distributions

Degree distribution is one of the fundamental properties of networks. Below, we plot the empirical cumulative distribution functions (cdf) of the degree distribution for the GitHib and Grid graphs. 

As we noted before, for the GitHub graph, most nodes have low degree, but a few have very high degree, up to almost 10,000. This is not at all the case with the Grid graph where almost all nodes having degree less than 10, and the maximum degree observed is only 16.


In [ ]:
## degree distribution - GitHub graph
deg = GitHubGraph.degree()
e = ecdf(deg) ## empirical cdf
x = np.arange(1,max(deg),1)
y = [e(i) for i in x]

## plot on log scale for x-axis
plt.semilogx(x,y,'-',color='black',label='GitHub')
plt.xlabel('degree',fontsize=14)
plt.ylabel('empirical cdf',fontsize=14);
plt.title('GitHub graph (ml and web developers)');

In [ ]:
## degree distribution - Grid graph
## we see much lower degrees in ths case
deg = Grid.degree()
e = ecdf(deg)
x = np.arange(1,30,1)
y = [e(i) for i in x]

## plot on linear scale for x-axis
plt.plot(x,y,'-',color='black',label='Grid')
plt.xlabel('degree',fontsize=14)
plt.ylabel('empirical cdf',fontsize=14);
plt.title('Power Grid Network');

## Shortest paths distribution

In the plots below, we consider 100 randomly chosen nodes, and compute the length of the shortest path to reach every other node. We then plot histograms for those values. 

Once again we see much different distributions: for the GitHub graph, most paths are quite short, with common values in the range from 2 to 6.
For the Grid graph however, the paths are generally longer, and over a much wider range.


In [ ]:
## shortest paths length from a given node, GitHub graph
sg = GitHubGraph.connected_components().giant() ## giant component, so every pair of nodes are connected
V = np.random.choice(sg.vcount(),size=100,replace=False) ## sample of vertices
sp = [x for y in sg.distances(source=V) for x in y if x>0] ## drop 0-distances (distance to self)
c = Counter(sp)
s = sorted(c.items())

## barplot
fig, ax = plt.subplots()
b = ax.bar([x[0] for x in s], [x[1] for x in s], color='darkgrey') 
ax.set_yscale('log')
ax.set_xlabel('path length',fontsize=14)
ax.set_ylabel('number of paths (log scale)',fontsize=14)
plt.title('GitHub graph (ml and web developers)');

In [ ]:
## min path length from that node to other nodes, Grid network
sg = Grid.connected_components().giant() ## giant component, so every pair of nodes are connected
V = np.random.choice(sg.vcount(),size=100,replace=False) ## sample
sp = [x for y in sg.distances(source=V) for x in y if x>0] ## drop 0-distances (distance to self)
c = Counter(sp)
s = sorted(c.items())

## barplot
fig, ax = plt.subplots()
b = ax.bar([x[0] for x in s], [x[1] for x in s], color='darkgrey') 
ax.set_xlabel('path length',fontsize=14)
ax.set_ylabel('number of paths',fontsize=14)
plt.title('Power Grid Network');

## Local clustering coefficient

Below, we compare the average local clustering coefficients as a function
of the node degrees for the GitHub graph. We consider degrees from 10 to 1000.

Looking at a log-log plot, we see a power law relation between those quantities; 
we also compute the regression line for comparison.


In [ ]:
## GitHub graph: build dataframe with degrees and local clustering coefficients
## and compute mean values w.r.t. degree.
df = pd.DataFrame(np.array([GitHubGraph.transitivity_local_undirected(mode='zero'),GitHubGraph.degree()]).transpose(),
            columns=['ClustCoef','degree'])
## keep row where degree is in range [10,1000]
mindeg = 10
maxdeg = 1000
df = df[(df.degree >= mindeg) & (df.degree <= maxdeg)]
df = df.groupby(by='degree').mean()
df.head()

In [ ]:
## fit a regression (log-log scale) and plot
X = [np.log(i) for i in df.index] ## degrees
Y = [np.log(i) for i in df.ClustCoef]
regressor = LinearRegression()  
regressor.fit(np.array(X).reshape(-1,1), Y)

## plot on log-log scale
b = regressor.intercept_
a = regressor.coef_[0]
print('power law exponent:',a)
plt.loglog(df.index,df.ClustCoef,'.-',color='grey')
## since log Y = a log X + b, Y = e^b X^a, we plot:
plt.plot([mindeg,maxdeg],[np.exp(b)*mindeg**a,np.exp(b)*maxdeg**a],color='black')
plt.xlabel('log (degree)',fontsize=14)
plt.ylabel('log (mean local clust. coef.)',fontsize=14)
plt.title('GitHub graph (ml and web developers)');
